In [1]:
%cd /nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/source

import sys
import os

sys.path.append("libs")

import ROOT

from libs.CNNjetTagger import CNN_jet_tagger


data_dir = "/nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/data/"

/nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/source
Welcome to JupyROOT 6.30/02


2024-04-03 13:38:24.787784: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = CNN_jet_tagger(input_type=False)
model.load_model("models/model_0")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 30, 30, 12, 1)]   0         
                                                                 
 conv3d_0 (Conv3D)           (None, 30, 30, 12, 30)    840       
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 15, 6, 30)     24330     
                                                                 
 batch_norm_0 (BatchNormali  (None, 15, 15, 6, 30)     120       
 zation)                                                         
                                                                 
 max_pool_0 (MaxPooling3D)   (None, 7, 7, 6, 30)       0         
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 4, 6, 30)       24330     
                                                             

In [3]:
def get_tchains(file_names_sig, file_names_bg, start_entries=[0,0], n_entries=[-1,-1], tree_name="T"):
    tchain_bg = ROOT.TChain(tree_name)
    for file_name in file_names_bg:
        tchain_bg.Add(file_name)
    
    tchain_sig = ROOT.TChain(tree_name)
    for file_name in file_names_sig:
        tchain_sig.Add(file_name)
    
    tchains_orig = [tchain_sig, tchain_bg]
    tchains = tchains_orig
    
    print(tchains[0].GetEntries(), tchains[1].GetEntries())
    
    if n_entries[0] != -1:
        #subchain_sig = tchain_sig.CopyTree("", nentries=n_entries[0], firstentry=start_entries[0])
        subchain_sig = tchain_sig.CloneTree(0)
        subchain_sig.CopyEntries(tchain_sig, nentries=n_entries[0])
        tchains[0] = subchain_sig
    if n_entries[1] != -1:
        #subchain_bg = tchain_bg.CopyTree("", nentries=n_entries[1], firstentry=start_entries[1])
        subchain_bg = tchain_bg.CloneTree(0)
        subchain_bg.CopyEntries(tchain_bg, nentries=n_entries[1])
        tchains[1] = subchain_bg
    
    print(tchains[0].GetEntries(), tchains[1].GetEntries())
    return tchains

tchains = get_tchains([data_dir+"flat/signal.root"], [data_dir+"flat/background2.root", data_dir+"flat/background3.root"], n_entries=[-1, 150000])

154943 7877808
154943 150000


In [4]:
#model.test_loading(tchains, 5000)
branches = ["GNNjet_pt", "GNNjet_eta", "GNNjet_EMfrac", "GNNjet_m"]
log_scale = [True, False, False, True]
scale = [0.001, 1, 1, 0.001]
names = ["pT [GeV]", "eta", "EMfrac", "m [GeV]"]
model.measure_performance(tchains, 5000, split_ratio=[0.6,0.2,0.2], plot_performance_evt_quantities=branches, log_scale=log_scale, scale=scale, names=names, plot_test_history=False, plot_train_val_history=True)

Data splitting updated!:
	split_ratio = [0.6, 0.2, 0.2] 	batch_size = 5000 	h5_blocksize = 200000
	train: signal = 92965 	background = 90000 	batch_size_signal = 2540 	batch_size_background = 2460 	n_batches = 36
	val: signal = 30988 	background = 30000 	batch_size_signal = 2540 	batch_size_background = 2460 	n_batches = 12
	test: signal = 30990 	background = 30000 	batch_size_signal = 2540 	batch_size_background = 2460 	n_batches = 12
	total: signal = 154943 	background = 150000 



Traceback (most recent call last):
  File "/cvmfs/sft.cern.ch/lcg/views/LCG_105/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22409/3851170428.py", line 6, in <module>
    model.measure_performance(tchains, 5000, split_ratio=[0.6,0.2,0.2], plot_performance_evt_quantities=branches, log_scale=log_scale, scale=scale, names=names, plot_test_history=False, plot_train_val_history=True)
  File "/nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/source/libs/CNNjetTagger.py", line 773, in measure_performance
    predictions = self.model.predict(batched_test_data, batch_size=batch_size, steps=self.data_splitting.n_batches_test)
  File "/cvmfs/sft.cern.ch/lcg/views/LCG_105/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/cvmfs/sft.cern.ch/lcg

In [ ]:
# asymmetric distributions
# EMfrac stops at 0.4 (+ cut EMfrac>0)
# 
# image preprocessing, flattening/weighting in pt, eta, ...
# plots for GNN
# slides status report